<a href="https://colab.research.google.com/github/aswinaus/MS-Copilot-Semantic-Index-API/blob/main/Copilot_Retrieval_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Summary** : Code in this notebook retrieves relevant documents from SharePoint using the Microsoft Graph Copilot Retrieval API and ranks them by relevance using reranking logic based on the semantic index score and then leverages OpenAI API to summarize the key information from those documents.

In [ ]:
import requests
import json
# Replace with your actual access token
from google.colab import userdata
GRAPH_TOKEN = userdata.get('GRAPH_TOKEN')
access_token = GRAPH_TOKEN

url = "https://graph.microsoft.com/beta/copilot/retrieval"

headers = {
  "Authorization": f"Bearer {access_token}",
  "Content-Type": "application/json"
}

request_body = {
  "queryString": "Please get me information about how many EYI MyDocs workspaces contains document about Netherlands workspace",
  "dataSource": "sharePoint",
  "resourceMetadata": [
    "title",
    "author"
  ],
  "maximumNumberOfResults": "10"
}

response = requests.post(url, headers=headers, data=json.dumps(request_body))

if response.status_code == 200:
  data = response.json()
  print("API Call Successful:")
  print(json.dumps(data, indent=2))
else:
  print(f"API Call Failed with status code: {response.status_code}")
  #print(response.text)

In [ ]:
if response.status_code == 200:
  data = response.json()
  print("API Call Successful:")

  # Rerank the results based on relevance score
  if "retrievalHits" in data:
    reranked_hits = sorted(data["retrievalHits"], key=lambda x: x.get("relevanceScore", 0), reverse=True)
    data["retrievalHits"] = reranked_hits
    print("Results reranked by relevance score.")

  print(json.dumps(data, indent=2))
else:
  print(f"API Call Failed with status code: {response.status_code}")
  #print(response.text)

In [ ]:
import os
from openai import OpenAI

# Get the OpenAI API key from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
except userdata.SecretNotFoundError:
    print("OpenAI API key not found in Colab secrets. Please add it.")
    OPENAI_API_KEY = None

if OPENAI_API_KEY:
    client = OpenAI()

    # Prepare the data to be sent to the OpenAI model
    # You can adjust the prompt and data format based on your needs
    prompt = "Now I want the results to be classified based on unique metadata identified in the results\n\n"
    for hit in data.get("retrievalHits", []):
        prompt += f"URL: {hit.get('webUrl', 'N/A')}\n"
        for extract in hit.get('extracts', []):
            prompt += f"Text: {extract.get('text', 'N/A')}\n\n"

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # You can change the model as needed
            messages=[
                {"role": "system", "content": "You are a helpful EYI MyDocs assistant that summarizes information."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500  # Adjust max_tokens as needed
        )
        print("\nResponse from LLM:")
        print(response.choices[0].message.content)

    except Exception as e:
        print(f"An error occurred while calling the OpenAI API: {e}")
else:
    print("Skipping OpenAI API call because the API key was not found.")